# Interfaces, Patterns, and Modularity

Design patterns:

* Are reusable solutions to many common design problems appearing in software engineering. 

* Are often language-agnostic and thus can be expressed using many programming languages.


### zope.interface

Consider the following code that checks if a list of geometric figures collide with the Axis-Aligned Bounding Box (AABB) algorithm. A simple implementation would be as follows:


In [1]:
!python _01_zope_interface/colliders_simple.py


(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))


But what if we add another figure that is missing some important attribute?. We'll only see the error at runtime:


In [2]:
from _01_zope_interface.colliders_simple import Point, Square, find_collisions

try:
    coll_point = find_collisions([
        Square(0, 0, 10),
        Point(100, 200),
    ])
except AttributeError as e:
    print(f'ERROR: {repr(e)}')

ERROR: AttributeError("'Point' object has no attribute 'bounding_box'")


We can avoid those kind of errors declaring explicit interfaces. The zope.interface allows us to create and implement explicit interfaces, as follows:

In [3]:
!python _01_zope_interface/colliders_interfaces.py


Valid attempt:

(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))

Invalid attempt (a detailed exception will be raised):

Traceback (most recent call last):
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_01_zope_interface/colliders_interfaces.py", line 106, in <module>
    for collision in find_collisions([
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_01_zope_interface/colliders_interfaces.py", line 27, in find_collisions
    verifyObject(ICollidable, item)
  File "/home/work/.cache/pypoetry/virtualenvs/expert-python-programming-fourth-edition-em7utuy--py3.10/lib/python3.10/site-packages/zope/interface/verify.py", line 166, in verifyObject
    return _verify(iface, candidate, tentative, vtype='o')
  File "/home/work/.cache/pypoetry/vi

In the last code example, the `verifyObject` function (there's also a similar `verifyClass` function) is used to raise a detailed exception if one of the class instances does not implement the interface `ICollidable` correctly.


The `verifyClass` and `verifyObject` functions only verify the surface area of the interface and aren't able to traverse into attribute types. 

We can do a more in-depth verification with the `validateInvariants` function, which checks if the class has all attributes necessary to satisfy the `ICollidable` interface, but also verifies whether the structure of those attributes satisfies constraints defined in the interface.

The new example is as follows:

In [4]:
!python _01_zope_interface/colliders_invariants.py


Valid attempt:

(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))

Invalid attempt (a detailed exception will be raised):

Traceback (most recent call last):
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_01_zope_interface/colliders_invariants.py", line 120, in <module>
    for collision in find_collisions([
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_01_zope_interface/colliders_invariants.py", line 27, in find_collisions
    ICollidable.validateInvariants(item)
  File "/home/work/.cache/pypoetry/virtualenvs/expert-python-programming-fourth-edition-em7utuy--py3.10/lib/python3.10/site-packages/zope/interface/interface.py", line 869, in validateInvariants
    invariant(obj)
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Pyt

In the last code example, the `Point` class seems to implement the `ICollidable` interface on the surface, bit it does not satisfy the interface invariant: it doesn't have the `bounding_box` property as instance of the `Box` type, so an exception is raised.


### Abstract Base Classes (ABCs)

In [5]:
some_type = []


In Python, type comparisons like the following example should be avoided:


In [6]:
assert type(some_type) == list


A better approach is to use the isinstance() function, so you can do type checking against a given type and also its subtypes, as follows: 



In [7]:
assert isinstance(some_type, list)


You can even use a range of types to check the type compatibility, as follows:


In [8]:
assert isinstance(some_type, (list, tuple, range))




But this method won't work if you create a custom class that defines the `__iter__()` method (so it behaves similar to a list) but inherits directly from `object`. This is where ABCs are useful.

ABC is a class that doesn't provide a concrete implementation, but defines a blueprint of a class that may be used to check against type compatibility.


The following shows a generic ABC, many incorrect implementations and a correct implementation of it:

In [9]:
!python _02_function_annotations_ABC/dummy_interface.py



Instantiating ABC: <class '__main__.DummyInterface'>
ERROR: TypeError("Can't instantiate abstract class DummyInterface with abstract methods dummy_method, dummy_property")

Instantiating ABC: <class '__main__.InvalidDummy'>
ERROR: TypeError("Can't instantiate abstract class InvalidDummy with abstract methods dummy_method, dummy_property")

Instantiating ABC: <class '__main__.MissingMethodDummy'>
ERROR: TypeError("Can't instantiate abstract class MissingMethodDummy with abstract method dummy_method")

Instantiating ABC: <class '__main__.MissingPropertyDummy'>
ERROR: TypeError("Can't instantiate abstract class MissingPropertyDummy with abstract method dummy_property")

Instantiating ABC: <class '__main__.Dummy'>
SUCCESS


From the last code examples:

* The `@abstractmethod` decorator indicates that a method must be implemented in classes that subclass the ABC. 

* If a class doesn't implement one of the abstract methods, a `TypeError` exception will be raised.

* This way you can ensure implementation completeness.


So the collider detection code can be modified to use ABCs, as follows:


In [10]:
!python _02_function_annotations_ABC/colliders_abc.py


Valid attempt:

(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))

Invalid attempt (with a dataclass that doesn't inherit from the ABC):

TypeError('Point(x=100, y=200) is not a collider')

Invalid attempt (with a dataclass that doesn't implement the ABC's abstract methods):

TypeError("Can't instantiate abstract class PointWithABC with abstract method bounding_box")


ABCs provide the special `__subclasshook__(cls)` method. We can use it to add validation logic to check whether a subclass of `ColliderABC` is implicitly compatible with it, as follows:


In [11]:
!python _02_function_annotations_ABC/colliders_subclasshooks.py


Valid attempt:

(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))

Invalid attempt:

Traceback (most recent call last):
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_02_function_annotations_ABC/colliders_subclasshooks.py", line 127, in <module>
    for collision in find_collisions([
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_02_function_annotations_ABC/colliders_subclasshooks.py", line 39, in find_collisions
    raise TypeError(f"{item} is not a collider")
TypeError: Point(x=100, y=200) is not a collider


With the `__subclasshook__()` method defined that way, `ColliderABC` becomes an implicit interface:

* Any object will be considered an instance of ColliderABC as long as it has the structure that passes the subclass hook check.

* So we can create classes compatible with the `ColliderABC` interface without explicitly inheriting from it.

The following example shows implicit interfaces in action:

In [15]:
from _02_function_annotations_ABC.colliders_subclasshooks import ColliderABC, Line, Point


line = Line(Point(0, 0), Point(100, 100))


In [16]:
line.bounding_box

Box(x1=0, y1=0, x2=100, y2=100)

In [17]:
isinstance(line, ColliderABC)

True